In [ ]:
#Project 1 - MLP
#Build MLP model to predict wine types based on 13 variables

######################################################################################
############################### REPORT ###############################################
# To implement the multi layer perceptron model I decided to use the lbfgs solver which 
# stands for limited-memory broyde-fletcher-goldfarb-shanno algorithm. this algorithm minimizes
# f(x) over unconstrained values of real-vector x where f is a differentiable scalar function
# I chose 3 different learning rates to compare and contrast the accuracies of each. As you
# can see by running the code, the highest learning rate has the highest accuracy. I also implemented
# 5-fold cross-validation using Kfold. This allows the data to be split into 5 subsets, each of which 
# is divided further into training and testing subsets. I very much enjoyed learning the intricacies
# of the MLP algorithm as well as how to adjust parameters in sklearn to achieve optimal results
######################################################################################


import numpy as np
import pandas as pd
import csv
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, train_test_split, cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt



wine = pd.read_csv('https://archive.ics.uci.edu/'
                      'ml/machine-learning-databases/wine/wine.data',
                      header=None)
# print(wine.head(5))

# wine = wine.drop([0])

# print(wine.head(5))

wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
                   'Proline']


# check if dataset has missing values
# print(wine.isnull().sum())

# Show a heatmap for highly correlated features. Something I found that I wanted to try
# cor = wine[wine.columns].corr()
# sns.heatmap(cor, cmap="YlGnBu", annot=True)

# Using the correlations, we could refine the data to have a more precise classifier
# drop attributes with correlation less than .6 with eachother
# wine = wine.drop(['proline', 'total_phenols', 'proanthocyanins', 'flavanoids', 'ash'], axis=1)

#seperate into training and testing data
X = wine.drop('Class label', axis=1)
Y = wine['Class label']


#normalize the X data
X = preprocessing.normalize(X, norm='l2', axis=0)

# # print(X[:10])

# # X_train, X_test, y_train, y_test = \
# #     train_test_split(X, y, test_size=0.3, random_state=0)
  
# seperate into training and testing using KFold (does the 5-fold cross-validation)
kf = KFold(n_splits = 5)
#verbose setting
v = False
#high learning rate
clf1 = MLPClassifier(solver='lbfgs', alpha=.05, hidden_layer_sizes=(5, 2), max_iter=1000, random_state=1, verbose=v)
#medium learning rate
clf2 = MLPClassifier(solver='lbfgs', alpha=.005, hidden_layer_sizes=(5, 2), max_iter=10000, random_state=1, verbose=v)
#low learning rate
clf3 = MLPClassifier(solver='lbfgs', alpha=.0005, hidden_layer_sizes=(5, 2), max_iter=10000, random_state=1, verbose=v)


# #function to train the different classifiers using 5-fold cross-validation
def trainMLP(classifier):
  for train, test in kf.split(X):
    #print("TRAIN:", train, "TEST:", test)
    classifier.fit(X[train], Y.iloc[train])
    print("training set accuracy: %f" % classifier.score(X[train], Y.iloc[train]))
    print("training set loss: %f" % classifier.loss_)
    classifier.predict(X[test])
    return classifier.score(X[test], Y.iloc[test]), classifier.loss_

#call the training function on the different types of classifiers and print accuracy
clf1_accuracy, clf1_loss = trainMLP(clf1)
print("clf1 accuracy: ", clf1_accuracy, " , clf1 loss: ", clf1_loss)


clf2_accuracy, clf2_loss = trainMLP(clf2)
print("clf2 accuracy: ", clf2_accuracy, " , clf2 loss: ", clf2_loss)

clf3_accuracy, clf3_loss = trainMLP(clf3)
print("clf3 accuracy: ", clf3_accuracy, " , clf3 loss: ", clf3_loss)



#random baseline prediction model
total_correct = 0
for entry in Y:
  guess = np.random.randint(1,3)
  if entry == guess:
    total_correct += 1
  

random_accuracy = total_correct / Y.size

print("random accuracy: %f" % random_accuracy)



##############################################################################################
# MLP from scratch
##############################################################################################

# Here I tried implementing MLP without sklearn and ran into some trouble
# In hindsight I wish I started this project earlier as I believe this was not as difficult 
# as I made it become

##############################################################################################


# Y = wine['Class label'].tolist()

# #Normalize Y data to be used
# for i in range(len(Y)):
#   if Y[i] == 1:
#     Y[i] = 0
#   if Y[i] == 2:
#     Y[i] = .5
#   if Y[i] == 3: 
#     Y[i] = 1

#   #add Output data back to inputs
# # X = X.tolist()
# # pos = 0
# # for x in X:
# #   x.append(Y[pos])
# #   pos += 1

# weights = [0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.075, 0.025, 0]

# class Layer:
#     def __init__(self):
#         pass
    
#     def forward(self, input):
#         return input
#     def backward(self, input, grad_output):
#         num_units = input.shape[1]
        
#         d_layer_d_input = np.eye(num_units)
        
#         return np.dot(grad_output, d_layer_d_input) # chain rule

# class ReLU(Layer):
#     def __init__(self):
#         pass
    
#     def forward(self, input):
#         relu_forward = np.maximum(0,input)
#         return relu_forward
    
#     def backward(self, input, grad_output):
#         # Compute gradient of loss w.r.t. ReLU input
#         relu_grad = input > 0
#         return grad_output*relu_grad

# class Dense(Layer):
#     def __init__(self, input_units, output_units, learning_rate=0.1):
        
#         self.learning_rate = learning_rate
#         self.weights = np.random.normal(loc=0.0, scale = np.sqrt(2/(input_units+output_units)), size = (input_units,output_units))
#         self.biases = np.zeros(output_units)
        
#     def forward(self,input):
        
#         return np.dot(input,self.weights) + self.biases
    
#     def backward(self,input,grad_output):
#         grad_input = np.dot(grad_output, self.weights.T)

#         grad_weights = np.dot(input.T, grad_output)
#         grad_biases = grad_output.mean(axis=0)*input.shape[0]
        
#         assert grad_weights.shape == self.weights.shape and grad_biases.shape == self.biases.shape

#         self.weights = self.weights - self.learning_rate * grad_weights
#         self.biases = self.biases - self.learning_rate * grad_biases
        
#         return grad_input

# def softmax_crossentropy_with_logits(logits,reference_answers):
#     logits_for_answers = logits[np.arange(len(logits)),reference_answers]
    
#     xentropy = - logits_for_answers + np.log(np.sum(np.exp(logits),axis=-1))
    
#     return xentropy
# def grad_softmax_crossentropy_with_logits(logits,reference_answers):
#     ones_for_answers = np.zeros_like(logits)
#     ones_for_answers[np.arange(len(logits)),reference_answers] = 1
    
#     softmax = np.exp(logits) / np.exp(logits).sum(axis=-1,keepdims=True)
    
#     return (- ones_for_answers + softmax) / logits.shape[0]

# # # normalize x
# # X_train = X_train.astype(float) / 255.
# # X_test = X_test.astype(float) / 255.
# # # we reserve the last 10000 training examples for validation
# # X_train, X_val = X_train[:-10000], X_train[-10000:]
# # y_train, y_val = y_train[:-10000], y_train[-10000:]

# kf = KFold(n_splits=5)
# for train, test in kf.split(X):

# # if flatten:
# #     X_train = X_train.reshape([X_train.shape[0], -1])
# #     X_val = X_val.reshape([X_val.shape[0], -1])
# #     X_test = X_test.reshape([X_test.shape[0], -1])
# return X_train, y_train, X_val, y_val, X_test, y_test
# X_train, y_train, X_val, y_val, X_test, y_test = load_dataset(flatten=True)


# network = []
# network.append(Dense(X_train.shape[1],100))
# network.append(ReLU())
# network.append(Dense(100,200))
# network.append(ReLU())
# network.append(Dense(200,10))
# def forward(network, X):
    
#     activations = []
#     input = X
#     # Looping through each layer
#     for l in network:
#         activations.append(l.forward(input))
#         # Updating input to last layer output
#         input = activations[-1]
    
#     assert len(activations) == len(network)
#     return activations
# def predict(network,X):
#     # Compute network predictions. Returning indices of largest Logit probability
#     logits = forward(network,X)[-1]
#     return logits.argmax(axis=-1)
# def train(network,X,y):
    
#     layer_activations = forward(network,X)
#     layer_inputs = [X]+layer_activations  #layer_input[i] is an input for network[i]
#     logits = layer_activations[-1]
    
#     # Compute the loss and the initial gradient
#     loss = softmax_crossentropy_with_logits(logits,y)
#     loss_grad = grad_softmax_crossentropy_with_logits(logits,y)
    
#     for layer_index in range(len(network))[::-1]:
#         layer = network[layer_index]
        
#         loss_grad = layer.backward(layer_inputs[layer_index],loss_grad) #grad w.r.t. input, also weight updates
        
#     return np.mean(loss)



# # Make a prediction with weights
# def predict(row, weights):
# 	activation = weights[0]
# 	for i in range(len(row)-1):
# 		activation += weights[i + 1] * row[i]
# 	return 1.0 if activation >= 0.0 else 0.0

#   #Predict
# for row in X:
#   print(row)
#   prediction = predict(row, weights)
#   print("Expected=%d, Predicted=%d" % (row[-1], prediction))

# # Split a dataset into k folds
# def cross_validation_split(dataset, n_folds):
# 	dataset_split = list()
# 	dataset_copy = list(dataset)
# 	fold_size = int(len(dataset) / n_folds)
# 	for i in range(n_folds):
# 		fold = list()
# 		while len(fold) < fold_size:
# 			index = randrange(len(dataset_copy))
# 			fold.append(dataset_copy.pop(index))
# 		dataset_split.append(fold)
# 	return dataset_split

# # Calculate accuracy percentage
# def accuracy_metric(actual, predicted):
# 	correct = 0
# 	for i in range(len(actual)):
# 		if actual[i] == predicted[i]:
# 			correct += 1
# 	return correct / float(len(actual)) 
 
# # Evaluate an algorithm using a cross validation split
# def evaluate_algorithm(dataset, algorithm, n_folds, *args):
# 	folds = cross_validation_split(dataset, n_folds)
# 	scores = list()
# 	for fold in folds:
# 		train_set = list(folds)
# 		train_set.remove(fold)
# 		train_set = sum(train_set, [])
# 		test_set = list()
# 		for row in fold:
# 			row_copy = list(row)
# 			test_set.append(row_copy)
# 			row_copy[-1] = None
# 		predicted = algorithm(train_set, test_set, *args)
# 		actual = [row[-1] for row in fold]
# 		accuracy = accuracy_metric(actual, predicted)
# 		scores.append(accuracy)
# 	return scores
 
# # Make a prediction with weights
# def predict(row, weights):
# 	activation = weights[0]
# 	for i in range(len(row)-1):
# 		activation += weights[i + 1] * row[i]
# 	return 1.0 if activation >= 0.0 else 0.0
 
# # Estimate Perceptron weights using stochastic gradient descent
# def train_weights(train, l_rate, n_epoch):
# 	weights = [0.0 for i in range(len(train[0]))]
# 	for epoch in range(n_epoch):
# 		for row in train:
# 			prediction = predict(row, weights)
# 			error = row[-1] - prediction
# 			weights[0] = weights[0] + l_rate * error
# 			for i in range(len(row)-1):
# 				weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
# 	return weights
 
# # Perceptron Algorithm With Stochastic Gradient Descent
# def perceptron(train, test, l_rate, n_epoch):
# 	predictions = list()
# 	weights = train_weights(train, l_rate, n_epoch)
# 	for row in test:
# 		prediction = predict(row, weights)
# 		predictions.append(prediction)
# 	return(predictions)
 
# # Test the Perceptron algorithm on the sonar dataset
# seed(1)
# # load and prepare data
# filename = 'sonar.all-data.csv'
# dataset = load_csv(filename)
# for i in range(len(dataset[0])-1):
# 	str_column_to_float(dataset, i)
# # convert string class to integers
# str_column_to_int(dataset, len(dataset[0])-1)
# # evaluate algorithm
# n_folds = 3
# l_rate = 0.01
# n_epoch = 500
# scores = evaluate_algorithm(dataset, perceptron, n_folds, l_rate, n_epoch)
# print('Scores: %s' % scores)
# print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

training set accuracy: 1.000000
training set loss: 0.041695
clf1 accuracy:  0.9166666666666666  , clf1 loss:  0.04169498579185668
training set accuracy: 1.000000
training set loss: 0.009439
clf2 accuracy:  0.9166666666666666  , clf2 loss:  0.009438990491108514
training set accuracy: 0.985915
training set loss: 0.037899
clf3 accuracy:  0.7222222222222222  , clf3 loss:  0.03789852657853675
random accuracy: 0.421348
